In [1]:
from __future__ import print_function
import json
import nltk
from nltk.corpus import stopwords
import pandas as pd
import spacy
import string
from google.cloud import firestore
import datetime

import json
import base64
from google.oauth2 import service_account
import os
from google.cloud import language_v1
import requests


import traceback

In [2]:
import pandas as pd
import numpy as np
from surprise import Reader

In [3]:
from surprise import Dataset

In [4]:
df = pd.read_csv("topical_chat.csv")
df

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper
...,...,...,...
188373,8628,"Wow, it does not seem like that long. Since I...",Surprised
188374,8628,"I havent seen that episode, I might google it...",Curious to dive deeper
188375,8628,I don't think I have either. That's an insane...,Curious to dive deeper
188376,8628,"I did, my little brother used to love Thomas ...",Happy


In [5]:
#Assign Numeric Values of 1 or 0 to Categorical Data: Happy, Neutral, Sad, Surprised, Curious to dive deeper (shortened to curious)
df['Curious'] = np.where(df['sentiment'].str.contains("Curious"), 1, 0)
df['Happy'] = np.where(df['sentiment'].str.contains("Happy"), 1, 0)
df['Sad'] = np.where(df['sentiment'].str.contains("Sad"), 1, 0)
df['Neutral'] = np.where(df['sentiment'].str.contains("Neutral"), 1, 0)
df['Surprised'] = np.where(df['sentiment'].str.contains("Surprised"), 1, 0)
df

,conversation_id,message,sentiment,Curious,Happy,Sad,Neutral,Surprised
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper,1,0,0,0,0
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper,1,0,0,0,0
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper,1,0,0,0,0
3,1,Google provides online related services and p...,Curious to dive deeper,1,0,0,0,0
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper,1,0,0,0,0
...,...,...,...,...,...,...,...,...
188373,8628,"Wow, it does not seem like that long. Since I...",Surprised,0,0,0,0,1
188374,8628,"I havent seen that episode, I might google it...",Curious to dive deeper,1,0,0,0,0
188375,8628,I don't think I have either. That's an insane...,Curious to dive deeper,1,0,0,0,0
188376,8628,"I did, my little brother used to love Thomas ...",Happy,0,1,0,0,0


In [6]:
#Number of messages in each conversation
df1 = df['conversation_id'].value_counts()
#Copy datafram to manipulate
manipulated = df.copy()


In [7]:
#labels each conversation with two distinct users
manipulated["User"] = ""
current_user = 1
for x in range(df1[1]):
    if (x % 2 == 0):
        manipulated['User'][x] = current_user
    else:
        manipulated['User'][x] = current_user+1
current_user = current_user + 2
for i in range(len(df.index)-1):
    if (df['conversation_id'][i] == df['conversation_id'][i+1]):
        pass
    else:
        for y in range(df1[df['conversation_id'][i+1]]):
            if (y % 2 == 0):
                manipulated['User'][i+1+y] = current_user
            else:
                manipulated['User'][i+1+y] = current_user+1
        current_user = current_user + 2

<ipython-input-7-4a1339a11532>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manipulated['User'][x] = current_user
<ipython-input-7-4a1339a11532>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manipulated['User'][x] = current_user+1
<ipython-input-7-4a1339a11532>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manipulated['User'][i+1+y] = current_user
<ipython-input-7-4a1339a11532>:18: SettingWithCopyWarning: 
A value is trying 

In [8]:
data = [manipulated["Curious"], manipulated["Happy"], manipulated["Sad"], manipulated["Neutral"], manipulated["Surprised"]]

headers = ["Curious", "Happy", "Sad", "Neutral", "Surprised"]

feature_target = pd.concat(data, axis=1, keys=headers)
feature_target[0:10]

,Curious,Happy,Sad,Neutral,Surprised
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
5,1,0,0,0,0
6,1,0,0,0,0
7,1,0,0,0,0
8,1,0,0,0,0
9,1,0,0,0,0


In [9]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T



In [11]:
dfsample = feature_target[0:25]


R = np.array(dfsample)
N = len(R)
M = len(R[0])
K = 5
P = np.random.rand(N,K)

Q = np.random.rand(M,K)
nP, nQ = matrix_factorization(R, P, Q, K)
nR = np.dot(nP, nQ.T)


In [12]:
nR

array([[1.03542846, 1.49313783, 0.95951111, 1.36275668, 1.1859734 ],
       [1.02732882, 0.96214459, 0.89391134, 1.13804448, 1.50713347],
       [1.01397922, 0.9403369 , 1.16467278, 1.4082489 , 1.16276066],
       [0.97488984, 1.36178302, 0.80020945, 1.21589911, 1.30654191],
       [1.01996381, 1.43100876, 0.92814486, 1.34895078, 1.38910176],
       [0.99072688, 1.46342696, 1.05792333, 1.47840912, 1.33506242],
       [1.0172065 , 1.32561878, 1.1569781 , 1.55811702, 1.40629428],
       [1.05644414, 1.69325716, 0.92062344, 1.4408763 , 1.26408592],
       [0.95857575, 1.41896053, 0.83446889, 1.18979858, 0.87812663],
       [1.01706902, 1.31825463, 0.90769824, 1.25767428, 1.27280214],
       [0.91670367, 0.85502082, 0.77191524, 1.00866048, 1.19989016],
       [0.92281946, 1.27929667, 0.96274137, 1.30742927, 1.06330271],
       [1.04813446, 1.21845512, 0.98935079, 1.33743306, 1.30522202],
       [0.94965711, 0.99368815, 1.09936553, 1.39002272, 1.47561553],
       [0.95972036, 1.24637233, 1.

In [13]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('brown')

def analyze_blob_sentiment(text):
    blob = TextBlob(str(text))
    return blob.sentiment.polarity

def analyze_blob_subjectivity(text):
    blob = TextBlob(str(text))
    return blob.sentiment.subjectivity

def analyze_vader_sentiment(text):
    vader_analyzer = SentimentIntensityAnalyzer()
    sentiment_dict = vader_analyzer.polarity_scores(text)
    return sentiment_dict['compound']

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mnaga\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mnaga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\mnaga\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [14]:
analyze_vader_sentiment("Are you a fan of Google or Microsoft?")

0.3182

In [42]:
manipulated = manipulated.sample(frac=1).reset_index(drop=True)
split = np.array_split(manipulated, 600)

split1=split[0].copy()

In [43]:
split1

,conversation_id,message,sentiment,Curious,Happy,Sad,Neutral,Surprised,User
0,6742,"I rarely watch football, but I am interested ...",Surprised,0,0,0,0,1,13484
1,6663,"Basketball drama is usually off the court, aw...",Neutral,0,0,0,1,0,13325
2,3274,Marcia Lucas won an award for editing. Well d...,Neutral,0,0,0,1,0,6548
3,6172,I got to be honest the older i get and as far...,Curious to dive deeper,1,0,0,0,0,12344
4,6749,But did you know soccer makes more money than...,Curious to dive deeper,1,0,0,0,0,13498
...,...,...,...,...,...,...,...,...,...
309,3763,A delicious three line button? Nice chatting ...,Happy,0,1,0,0,0,7526
310,2726,Oh smart thatll make everyone want to see the...,Neutral,0,0,0,1,0,5451
311,1546,That is so nice! I miss that game.,Neutral,0,0,0,1,0,3091
312,7084,Yeah he said because his belief system preven...,Surprised,0,0,0,0,1,14167


In [45]:
url = "https://us-central1-treehacks-karmalink.cloudfunctions.net/NLPAnalysis"
split1
split2 = split1.copy()

In [47]:



for i, row in split[0].iterrows():

    
    split2.at[i, 'Vader Sentiment Score'] = analyze_vader_sentiment(split[0]['message'].values[i])

    split2.at[i, 'Blob Subjectivity'] = analyze_blob_subjectivity(split[0]['message'].values[i])
    
    param = {"message": split[0]['message'].values[i]}
    r1 = requests.post(url, json=param)
    http_response1 = json.loads(r1.text)
    split2.at[i, 'Cloud Polarity'] = http_response1['Cloud_Polarity']
    split2.at[i, 'TextBlob Polarity']= http_response1['TextBlob_Polarity']
    
print(split2)
    


     conversation_id                                            message  \
0               6742   I rarely watch football, but I am interested ...   
1               6663   Basketball drama is usually off the court, aw...   
2               3274   Marcia Lucas won an award for editing. Well d...   
3               6172   I got to be honest the older i get and as far...   
4               6749   But did you know soccer makes more money than...   
..               ...                                                ...   
309             3763   A delicious three line button? Nice chatting ...   
310             2726   Oh smart thatll make everyone want to see the...   
311             1546                That is so nice! I miss that game.    
312             7084   Yeah he said because his belief system preven...   
313             4178   Did you know Iceland has no public rail service?   

                   sentiment  Curious  Happy  Sad  Neutral  Surprised   User  \
0                  

In [49]:
factorization_data = [split2["Vader Sentiment Score"], split2["Blob Subjectivity"], split2['Cloud Polarity'], split2['TextBlob Polarity']]

headers1 = ["Vader Sentiment Score", "Blob Subjectivity", 'Cloud Polarity', 'TextBlob Polarity']

feature_target1 = pd.concat(factorization_data, axis=1, keys=headers1)
feature_target1[0:30]


,Vader Sentiment Score,Blob Subjectivity,Cloud Polarity,TextBlob Polarity
0,-0.8043,0.573333,-0.5,-0.140000
1,0.0000,0.250000,-0.1,-0.250000
2,0.4497,0.000000,0.0,0.000000
3,-0.3400,0.655556,-0.2,0.244444
4,0.0000,0.500000,0.0,0.500000
5,0.5562,0.475000,-0.2,0.266667
6,0.6249,0.480000,0.0,-0.240000
7,0.6486,0.708333,0.0,-0.133333
8,0.7473,0.600000,0.4,0.600000
9,0.4145,0.600000,-0.2,-0.033333


In [ ]:
R2 = np.array(feature_target1)

N2 = len(feature_target1)
M2 = len(feature_target1.iloc[0])
K2 = 2

P2 = np.random.rand(N2,K2)
Q2 = np.random.rand(M2,K2)

nP2, nQ2 = matrix_factorization(R2, P2, Q2, K2)
nR2 = np.dot(nP2, nQ2.T)


In [50]:

feature_target1['User'] = split2['User']
feature_target1

,Vader Sentiment Score,Blob Subjectivity,Cloud Polarity,TextBlob Polarity,User
0,-0.8043,0.573333,-0.5,-0.140000,13484
1,0.0000,0.250000,-0.1,-0.250000,13325
2,0.4497,0.000000,0.0,0.000000,6548
3,-0.3400,0.655556,-0.2,0.244444,12344
4,0.0000,0.500000,0.0,0.500000,13498
...,...,...,...,...,...
309,0.8856,0.733333,0.8,0.708333,7526
310,0.9055,0.580952,0.9,0.204762,5451
311,0.4668,0.700000,0.4,0.175000,3091
312,0.8270,0.500000,-0.3,0.500000,14167


In [51]:
X = feature_target1.iloc[:, :-1].values
y = feature_target1.iloc[:, 4].values
y=y.astype('int')
y

array([13484, 13325,  6548, 12344, 13498,  7092, 11232,  3060,  6818,
        2685, 12341, 12819, 10440,  9520, 16643,  7989, 14730, 10324,
        4295, 11852,  2268,  1179, 11586, 15631,  9303, 13963,   928,
       11199, 12286,   939, 11261, 14356,  4684,  6005, 12489, 17069,
        6239,  3271,  1562,  3913,  1350,  1683,  3786, 10189,  5588,
        2872, 11290,  8491, 12698, 11057,  9491,  7894, 12225, 12357,
       16313,  4776, 16128,  8810,  5408, 11339, 10677, 15883, 16575,
       11363,   449,  7422,    51, 10333,  1466,   520,  6447,  9692,
       12256, 16992,  9523,  6332, 13940, 10443, 12813,  1499,  1897,
        9772,  3312, 12615,  4814, 12867,  2033, 15831,  6205, 14882,
        2526,  4263,  2588, 10099,  3921, 13921,  6732,  3402,  7149,
        8268,  2730, 11928,  5123, 16311, 13875,  6941, 15775, 14456,
        2283,  7611,  4385, 11838,    63,  8731, 11336, 11320,  9318,
       13889, 14777, 14975,  9205, 12454, 14472, 10536,  8792, 14196,
         675, 16139,

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50)
y_test

array([  449,  9523,  1159, 10417, 10387,   928,  3101,  7056,  4196,
       10259,  7149,  2588,  3913, 11232,  8356, 12615, 17062, 14415,
       10189, 10079, 15631,  9680, 12041, 11926, 16575,  2728,   988,
       15545,  7299,  3271,  2760,  9896,  2730,  6447, 12454,  5140,
         676, 13560, 12585,  2913,  2283, 12742, 11505, 11803, 13889,
        3986,  2269, 11320,  1466,  5748,  4263,  9221,  5007,  5123,
        5922,  8805, 14777,  7441, 10983, 13119, 10035,  2268,  6096,
       12489, 15356, 16289, 17073,  2526,  7005, 12492, 11200,  9520,
        4862, 14653, 10440, 10821, 13595, 16779,   244,  2332,   675,
        3060,  9977,  7231,  4394,  7536, 14730,  9205, 12721,  1391,
        6110,  4383,  6732, 10705, 15026,  6818,  5577,  9983, 14009,
       12698,  7894,  2148,  9494,  9772,  2980, 10333,  5025, 12819,
        9318, 13764, 17061,  7989, 17069, 14882,  9618,  2073,  1683,
        7823, 13940,  7668, 14495,  6157,    51,  3528,  4008,  3662,
        8731,  2872,

In [ ]:
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel = 'linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
from sklearn import metrics
print(y_pred)

In [ ]:
print(feature_target1[feature_target1['User']== 9754].index.values)
print(feature_target1[feature_target1['User']== 10734].index.values)

In [ ]:
print(feature_target1['Vader Sentiment Score'][801])
print(feature_target1['Blob Subjectivity'][801])

In [ ]:
print(feature_target1['Vader Sentiment Score'][1125])
print(feature_target1['Blob Subjectivity'][1125])

In [53]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)
y_pred1 = classifier.predict(X_test)
y_pred1

array([ 7092,  6161, 10536,  2232,  6161, 12143,  4295,  1179,  1686,
        6650,  9491,  6161,  5408,  6255,  6161,  3786,  5554,  2264,
        2279,  3554,   751,  7092,   520,  7164,  2831,  6205,  5408,
        8491,   939,  7568,  2831,  5436,  2232,  2831,  1686,  8491,
        2004,  4385, 11164,  1897,  4037,   751,  3402,   905,  5451,
        6161,  1897,   751,  3033,   520,  3402,  2279,  4295,   516,
        6548,   905,  2685,  1897,  8491,  2232,  8237,   751,  4037,
        1686,  4295,  6161,  3786,  7164,  6431,  6332,  1179,  6161,
        1897,  3554,  3402,  2232,  2232,  7092,  3033,  1686,  1897,
        6255,  3402,  6205,  1897,  7164,  2279,  5436,  6066,  2033,
        7164,  3554,  4707,   516,  3554,  3786,  5408,  5436,  2685,
        4295,  4385,  1686,  8810,  8656,  2685,  6332,  6650,  3554,
        4295,   939,   905,  3786,  2004,  3402,  1897,  2279,  4501,
       10536,   905,  4037,  2831,  3402,  2279,   939,   939,  6205,
        3402,  3402,

In [ ]:
print(feature_target1[feature_target1['User']== 3283].index.values)

In [ ]:
print(feature_target1['Vader Sentiment Score'][52])
print(feature_target1['Blob Subjectivity'][52])

In [54]:
import joblib

# Export the model to a file
joblib.dump(classifier, 'model.joblib')

['model.joblib']

In [ ]:
X_test[20]

In [ ]:
unique_users = pd.DataFrame(feature_target1['User'].unique())
data1 = unique_users[0].tolist()


#unique_data = pd.concat(data1, axis=1, names=['Series name'])
#unique_data.columns =['Unique Users']
#len(unique_data)

unique_data['new'] = 0

data1[0:10]


In [ ]:

messages = []
for b in range(unique_users.shape[0]):
    messages.append(manipulated[manipulated['User'] == unique_users[0][b]]['message'])
messages


testframe = pd.DataFrame({'User': pd.Series([], dtype='int'),
                   'b': pd.Series([], dtype='object')})
testframe['User'] = unique_data['Unique Users']
testframe['b'] = messages
testframe

In [ ]:
messages_dict = dict(zip(data1, messages))

In [ ]:

def analyze_cloud_sentiment(text_phrase):
    # Defines the client, type of document, and the language used in the sentiment analysis
    client = language_v1.LanguageServiceClient()
    language = "en"
    document = language_v1.Document(content=text_phrase, type_=language_v1.Document.Type.PLAIN_TEXT, language=language)
    response = client.analyze_sentiment(request={'document': document})
    # Uses the analyze_sentiment() method to find the overall polarity and magnitude of the text
    text_polarity = response.document_sentiment.score
    return(text_polarity)

In [ ]:
analyze_cloud_sentiment("sup bruv")

In [ ]:
url = "https://us-central1-treehacks-karmalink.cloudfunctions.net/NLPAnalysis"
param = {"message": "sup bruv"}
r = requests.post(url, json=param)



In [87]:
lists1 = X_test.tolist()
json_string = json.dumps(lists1)
lists2 = y_test.tolist()
json_string2 = json.dumps(lists2)
res = {"X_test": json_string, "y_test": json_string2}

In [101]:
with open('data.json', 'w') as fp:
    res2 = json.dumps(res)

In [89]:
arr1 = np.asarray(json_string)

In [91]:
res

{'X_test': '[[0.5267, 0.1, -0.4000000059604645, 0.3], [-0.836, 0.0, -0.5, 0.0], [0.2924, 0.675, 0.0, 0.525], [-0.6467, 0.2, 0.20000000298023224, -0.03125], [-0.765, 0.0, -0.5, 0.0], [0.7351, 1.0, 0.10000000149011612, 0.5], [0.6705, 1.0, 0.8999999761581421, 1.0], [0.0, 0.1, -0.30000001192092896, -0.1], [0.658, 0.3972222222222222, 0.20000000298023224, 0.3333333333333333], [0.0, 0.42857142857142855, 0.10000000149011612, 0.21428571428571427], [0.0, 0.2, -0.30000001192092896, 0.2], [-0.296, 0.1875, -0.5, -0.0625], [0.7184, 0.725, 0.0, 0.2916666666666667], [0.6249, 0.4800000000000001, 0.0, -0.24000000000000005], [-0.296, 0.06666666666666667, -0.6000000238418579, 0.0], [0.8074, 0.6488095238095238, 0.5, 0.41071428571428575], [0.8103, 0.5734126984126985, 0.0, 0.10714285714285714], [0.2484, 0.6933333333333334, -0.20000000298023224, 0.08166666666666664], [0.165, 0.6575, -0.10000000149011612, 0.22749999999999998], [0.1154, 0.3914285714285714, 0.10000000149011612, 0.16], [-0.1481, 1.0, 0.3000000119

In [103]:
url1 = "https://us-central1-treehacks-karmalink.cloudfunctions.net/PredictUser"
param2 = res2
r3 = requests.post(url1, json=param2)

In [104]:
r3.text

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'

In [56]:
import googleapiclient.discovery

# Fill in your PROJECT_ID, VERSION_NAME and MODEL_NAME before running
# this code.

PROJECT_ID = "treehacks-karmalink"
VERSION_NAME = "Version_2"
MODEL_NAME = "Sentiment_KNN"

service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}'.format(PROJECT_ID, MODEL_NAME)
name += '/versions/{}'.format(VERSION_NAME)

# Due to the size of the data, it needs to be split in 2
first_half = X_test[:int(len(test_features)/2)]
second_half = X_test[int(len(test_features)/2):]

complete_results = []
for data in [first_half, second_half]:
    responses = service.projects().predict(
        name=name,
        body={'instances': data}
    ).execute()

    if 'error' in responses:
        print(response['error'])
    else:
        complete_results.extend(responses['predictions'])

# Print the first 10 responses
for i, response in enumerate(complete_results[:10]):
    print('Prediction: {}\tLabel: {}'.format(response, test_labels[i]))

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started